# MESH


In [ ]:
%%writefile GPU_SA.cu
#include "core.h"

int h_table1hop[17][17] = 
{{0,  0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15}};

int h_tablemesh[17][17] =
{{0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15},  
 {0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16},  
 {1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17},  
 {2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18},  
 {3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19},  
 {4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20},  
 {5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21},  
 {6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22},  
 {7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23},  
 {8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24},  
 {9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25},  
 {10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26},  
 {11,12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27},  
 {12,13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28},  
 {13,14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29},  
 {14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30},  
 {15,16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}};


int gridCost(int *positions){

    int cost_ = 0, increment=0, distManhattanI, distManhattanJ;

    for(int k=0; k<edges; k++){
        int ifrom = positions[h_edgeA[k]]/dim; 
        int jfrom = positions[h_edgeA[k]]%dim; 
        int ito = positions[h_edgeB[k]]/dim; 
        int jto = positions[h_edgeB[k]]%dim;
        
        distManhattanJ = abs(jto - jfrom);
        distManhattanI = abs(ito - ifrom);

        increment = h_tablemesh[distManhattanI][distManhattanJ];
        
        cost_ += increment;
    }

    return cost_;
}

void readPlacements(const char* fileName) {

    std::string name(fileName); // Without this line, the input mac_inout gives segfault..dunno why
    std::ifstream file(name, std::ifstream::in); 
    if(!file.good()) {
        printf("Failed to load random placements. Check file name or it's existence\nAborting");
        exit(EXIT_FAILURE);
    }

    std::string s;
    int* temp = (int*) malloc(sizeof(int) * gridSize);
    int i = 0;
    int start = 0;
    numGrids = 0;
    while(std::getline(file, s) && numGrids < MAX_GRIDS ) {
        
		std::stringstream line(s);
        int n;
        
        // Read placement line and store in @h_randomPlacements
        while(line >> n) h_randomPlacements[i++] = n;

        // Evaluating cost
        int positions[256];
        for(int k=start, l=0; k<i; k++, l++) {
            temp[l] = h_randomPlacements[k];
            positions[temp[l]] = l;
        }

        // Updating cost for 'ith' grid
        h_costs[numGrids] = gridCost(positions);
        start = i;
        numGrids++;
    }

    file.close();
}


__global__ void annealing(int nodes, int dim, int gridSize,              		// Variables
			   			  	int *d_grid,                        			 	// Initializers 
			   				int *v_i, int *v, int *A,                			// Graph 
                            int *d_results, int *d_costs, float* d_random,   	// Output and random states
                            int *table1hop, int *tablemesh               		// New cost parameters
                         ){		     

    
    // Determinig thread ID and checking if it's within boudaries
    int tId = blockIdx.x * blockDim.x + threadIdx.x;

	if( tId >= MAX_GRIDS ) return; 

    // Which sector of d_grid belongs to the thread
    size_t t_istart = tId*gridSize;
    float cuda_random;

    int currentCost = d_costs[tId], nextCost, increment, distManhattanI, distManhattanJ;
	
    int* localGrid = 		(int*) malloc(sizeof(int) * gridSize);
	int* localPositions = 	(int*) malloc(sizeof(int) * 256);
    
    for(int i=0; i<gridSize; i++) {
        localGrid[i] = d_grid[t_istart + i];
        localPositions[localGrid[i]] = i;
    }

    /* Simulated Annealing */
    unsigned int rndIndex = tId; 	// Each thread starts in a different place in @d_random
    bool foundOptimal = false;
    double T = 100;
    while(T>=0.00001 && !foundOptimal){

        for(int i=0; i<gridSize; i++){

                if(foundOptimal) break;

                for(int j=i+1; j<gridSize; j++){

					if(currentCost==0) {
						foundOptimal = true;
						break;
					}

                    // If we're looking at 2 empty spaces, skip                   
                    if(localGrid[i]==255 && localGrid[j]==255) continue;

                    int node1 = localGrid[i];
                    int node2 = localGrid[j];
                    
					nextCost = currentCost;
                    
					// Remove cost from node to it's neighbors
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){
                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim;
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = tablemesh[distManhattanI*17 + distManhattanJ];
                            nextCost -= increment;
                            
                        }
                        
                    }
                    if(node2!=255){
                        
                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = tablemesh[distManhattanI*17 + distManhattanJ];
                            nextCost -= increment;
                        }

                    }

                    // Swap nodes
                    int old1 = i;
                    int old2 = j;

                    if(node1!=255) localPositions[node1] = old2;
                    if(node2!=255) localPositions[node2] = old1;

                    localGrid[j] = node1;
                    localGrid[i] = node2;

                    // Recalculate cost to each neighbor based on the node's new position
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){

                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = tablemesh[distManhattanI*17 + distManhattanJ];
                            nextCost += increment;
                        }

                    }
                    if(node2!=255){

                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = tablemesh[distManhattanI*17 + distManhattanJ];
                            nextCost += increment;
                        }

                    }  

                    // Annealing acceptance criteria
                    double valor = exp(-1*(nextCost - currentCost)/T);
                    rndIndex = (rndIndex>=MAX_RANDS)?tId:rndIndex;
                    cuda_random = d_random[rndIndex];
                    rndIndex++;
  
                    if(nextCost <= currentCost || cuda_random <= valor)
                        currentCost = nextCost;
                    else {
                        if(node1!=255) localPositions[node1] = old1;
                        if(node2!=255) localPositions[node2] = old2;
                        localGrid[j] = node2;
                        localGrid[i] = node1;
                    }
                }

                T*=0.999;
            }
    }

    d_costs[tId] = currentCost;
	for(int i=0; i<gridSize; i++) d_results[t_istart+i] = localGrid[i];
}


void GPU_results(const char* fileName) {

    char placements[1000]; 
    strcpy(placements, fileName);
    strcat(placements, "_SAmesh_GPU_1k.out");

    std::ofstream output(placements, std::ofstream::out);

    int temp1Best[gridSize] = {0};
    int temp2[gridSize] = {0};
    
    int best = 100000;
    int tid = 0;
    int thread = 0;
    int pos = 0;

    for(int i=0; i<numGrids*gridSize; i++){ 

		if(i%gridSize==0 && i!=0) {

            pos = 0;
            if(h_costs[thread] < best) {
                best = h_costs[thread];
                for(int i=0; i<gridSize; i++) temp1Best[i] = temp2[i];
                tid = thread;
            }

            thread++;
            output << "\n";
        }

            temp2[pos++] = h_results[i];
            output << unsigned(h_results[i]) << " ";
    }

    output.close();

    // Best cost found
    std::cout << placements << "\t" << h_costs[tid] << "\t";
    for(int i=0; i<gridSize; i++){

        if(i%dim==0)
            std::cout << "\n" << temp1Best[i] << "\t";
        else
            std::cout << temp1Best[i] << "\t";

    }
    std::cout << "\n";

}

int main(int argc, const char **argv) {

    double timer = 0;
    cudaDeviceProp device;
    cudaGetDeviceProperties(&device, 0);

    // Pre-calculating random numbers (0,1]
	curandGenerator_t gen;
    cudaMalloc(&d_random, sizeof(float) * MAX_RANDS);
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerateUniform(gen, d_random, MAX_RANDS);
    
    h_costs = (int*) malloc(sizeof(int) * MAX_GRIDS);
    h_table1hop_1d = (int*) malloc(sizeof(int) * 17*17);
    h_tablemesh_1d = (int*) malloc(sizeof(int) * 17*17);

    int idx = 0;
    for(int i=0; i<17; i++)
        for(int j=0; j<17; j++){

            h_table1hop_1d[idx] = h_table1hop[i][j];
            h_tablemesh_1d[idx] = h_tablemesh[i][j];

            idx++;

        }

    // Read Graph
    readInput(argv[1]);
    std::cerr << "FILE: " << argv[3] << " running on " << device.name << std::endl;
    
	// Converting vector A to array
	h_A = (int*) malloc(sizeof(int) * A.size()); 
	memcpy(h_A, A.data(), sizeof(int) * A.size());

    // Read it's random placements (1k shuffles)
    readPlacements(argv[2]);

    // Allocating GPU memory
	cudaMalloc(&d_A,   		 sizeof(int)   * A.size());
	cudaMalloc(&d_v,   		 sizeof(int)   * nodes);
	cudaMalloc(&d_v_i, 		 sizeof(int)   * nodes);
    cudaMalloc(&d_seeds,     sizeof(float) * MAX_RANDS);
    cudaMalloc(&d_table1hop, sizeof(int)   * 17*17);
    cudaMalloc(&d_tablemesh, sizeof(int)   * 17*17);

	// Getting data from host to device
	cudaMemcpy(d_A, h_A,                    sizeof(int) * A.size(), cudaMemcpyHostToDevice); 
	cudaMemcpy(d_v, v, 	                    sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_v_i, v_i,                  sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_table1hop, h_table1hop_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_tablemesh, h_tablemesh_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);

	// Device SETUP
	size_t placements_size = MAX_BYTES * MAX_GRIDS; 
	
    // Host variables memory allocation
	h_results =   (int*) malloc(sizeof(int) * placements_size);
	
	// Allocating device memory
    cudaMalloc(&d_results,	sizeof(int) * placements_size); 
	cudaMalloc(&d_grid, 	sizeof(int) * placements_size);
    cudaMalloc(&d_costs, 	sizeof(int) * MAX_GRIDS);
   
   	// Getting data from host random_placements 
    cudaMemcpy(d_grid, h_randomPlacements,  sizeof(int) * placements_size,  cudaMemcpyHostToDevice);
    cudaMemcpy(d_costs, h_costs,            sizeof(int) * MAX_GRIDS,        cudaMemcpyHostToDevice);

   
    std::chrono::time_point<std::chrono::high_resolution_clock> start_clock, end_clock;
    start_clock = std::chrono::high_resolution_clock::now();

    annealing <<< MAX_BLOCKS, MAX_THREADS >>> ( nodes, dim, gridSize, 
                                                d_grid, d_v_i, d_v, d_A, 
                                                d_results, d_costs, d_random,
                                                d_table1hop, d_tablemesh
                                            );

    cudaDeviceSynchronize();

    cudaMemcpy(h_results, d_results,    sizeof(int) * placements_size,  cudaMemcpyDeviceToHost);
    cudaMemcpy(h_costs, d_costs,        sizeof(int) * MAX_GRIDS,        cudaMemcpyDeviceToHost);

    end_clock = std::chrono::high_resolution_clock::now();
    timer = std::chrono::duration_cast<Seconds>(end_clock - start_clock).count();

	GPU_results(argv[3]);
    printf("%.3lf\n", timer);    
}

/* Useful to know runtime specs
std::cerr << endl << "FILE: " << argv[3] << std::endl;
std::cerr << "BLOCKS: " << MAX_BLOCKS << std::endl;
cudaDeviceProp device;
cudaGetDeviceProperties(&device, 0);
printf("DEVICE: %s\nMAX_BLOCKS: %d\nMAX_THREADS: %d\n", device.name, device.maxGridSize[0], device.maxThreadsDim[0]);    
printf("RUNTIME: %.3lfs\n\n", timer);
*/

##Checking GPU data

In [ ]:
!nvcc GPU_SA.cu -lcurand
!mkdir GPU_results 2>/dev/null || true

!echo "Running micro instances"
!for file in nvcc_colab_icpp2020/list_micro/*.in; do ./a.out $file random_placement/micro/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> micro_mesh.txt; done
!mkdir GPU_results/mesh_micro 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_micro

!echo "Running kmeans_exp instances"
!for file in nvcc_colab_icpp2020/list_kmeans_expanded/*.in; do ./a.out $file random_placement/kmeans_exp/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_exp_mesh.txt; done
!mkdir GPU_results/mesh_kmeans_exp 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_kmeans_exp

!echo "Running cgrame instances"
!for file in nvcc_colab_icpp2020/list_cgrame/*.in; do ./a.out $file random_placement/cgrame/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> cgrame_mesh.txt; done
!mkdir GPU_results/mesh_cgrame 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_cgrame

!echo "Running kmeans instances"
!for file in nvcc_colab_icpp2020/list_kmeans/*.in; do ./a.out $file random_placement/kmeans/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_mesh.txt; done
!mkdir GPU_results/mesh_kmeans 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_kmeans

!echo "Running tree instances"
!for file in nvcc_colab_icpp2020/list_tree/*.in; do ./a.out $file random_placement/tree/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> tree_mesh.txt; done
!mkdir GPU_results/mesh_tree 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_tree

!echo "Running map instances"
!for file in nvcc_colab_icpp2020/list_map/*.in; do ./a.out $file random_placement/map/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> map_mesh.txt; done
!mkdir GPU_results/mesh_map 2>/dev/null || true
!mv *SAmesh_GPU_1k.out GPU_results/mesh_map

#1HOP

In [ ]:
%%writefile GPU_SA.cu
#include "core.h"

int h_table1hop[17][17] = 
{{0,  0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15}};

int h_tablemesh[17][17] =
{{0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15},  
 {0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16},  
 {1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17},  
 {2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18},  
 {3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19},  
 {4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20},  
 {5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21},  
 {6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22},  
 {7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23},  
 {8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24},  
 {9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25},  
 {10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26},  
 {11,12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27},  
 {12,13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28},  
 {13,14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29},  
 {14,15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30},  
 {15,16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}};


int gridCost(int *positions){

    int cost_ = 0, increment=0, distManhattanI, distManhattanJ;
    for(int k=0; k<edges; k++){
        int ifrom = positions[h_edgeA[k]]/dim; 
        int jfrom = positions[h_edgeA[k]]%dim; 
        int ito = positions[h_edgeB[k]]/dim; 
        int jto = positions[h_edgeB[k]]%dim;
        
        distManhattanJ = abs(jto - jfrom);
        distManhattanI = abs(ito - ifrom);

        increment = h_table1hop[distManhattanI][distManhattanJ];
        
        cost_ += increment;
    }
    return cost_;

}

void readPlacements(const char* fileName) {

    std::string name(fileName); // Without this line, the input mac_inout gives segfault..dunno why
    std::ifstream file(name, std::ifstream::in); 
    if(!file.good()) {
        printf("Failed to load random placements. Check file name or it's existence\nAborting");
        exit(EXIT_FAILURE);
    }

    std::string s;
    int* temp = (int*) malloc(sizeof(int) * gridSize);
    int i = 0;
    int start = 0;
    numGrids = 0;
    while(std::getline(file, s) && numGrids < MAX_GRIDS ) {
        
		std::stringstream line(s);
        int n;
        
        // Read placement line and store in @h_randomPlacements
        while(line >> n) h_randomPlacements[i++] = n;

        // Evaluating cost
        int positions[256];
        for(int k=start, l=0; k<i; k++, l++) {
            temp[l] = h_randomPlacements[k];
            positions[temp[l]] = l;
        }

        // Updating cost for 'ith' grid
        h_costs[numGrids] = gridCost(positions);
        start = i;
        numGrids++;
    }

    file.close();
}


__global__ void annealing(int nodes, int dim, int gridSize,              		// Variables
			   			  	int *d_grid,                        			 	// Initializers 
			   				int *v_i, int *v, int *A,                			// Graph 
                            int *d_results, int *d_costs, float* d_random,   	// Output and random states
                            int *table1hop, int *tablemesh               		// New cost parameters
                         ){		     

    
    // Determinig thread ID and checking if it's within boudaries
    int tId = blockIdx.x * blockDim.x + threadIdx.x;

	if( tId >= MAX_GRIDS ) return; 

    // Which sector of d_grid belongs to the thread
    size_t t_istart = tId*gridSize;
    float cuda_random;

    int currentCost = d_costs[tId], nextCost, increment, distManhattanI, distManhattanJ;
	
    int* localGrid = 		(int*) malloc(sizeof(int) * gridSize);
	int* localPositions = 	(int*) malloc(sizeof(int) * 256);
    
    for(int i=0; i<gridSize; i++) {
        localGrid[i] = d_grid[t_istart + i];
        localPositions[localGrid[i]] = i;
    }

    /* Simulated Annealing */
    unsigned int rndIndex = tId; 	// Each thread starts in a different place in @d_random
    bool foundOptimal = false;
    double T = 100;
    while(T>=0.00001 && !foundOptimal){

        for(int i=0; i<gridSize; i++){

                if(foundOptimal) break;

                for(int j=i+1; j<gridSize; j++){

					if(currentCost==0) {
						foundOptimal = true;
						break;
					}

                    // If we're looking at 2 empty spaces, skip                   
                    if(localGrid[i]==255 && localGrid[j]==255) continue;

                    int node1 = localGrid[i];
                    int node2 = localGrid[j];
                    
					nextCost = currentCost;
                    
					// Remove cost from node to it's neighbors
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){
                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim;
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = table1hop[distManhattanI*17 + distManhattanJ];
                            nextCost -= increment;
                            
                        }
                        
                    }
                    if(node2!=255){
                        
                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = table1hop[distManhattanI*17 + distManhattanJ];
                            nextCost -= increment;
                        }

                    }

                    // Swap nodes
                    int old1 = i;
                    int old2 = j;

                    if(node1!=255) localPositions[node1] = old2;
                    if(node2!=255) localPositions[node2] = old1;

                    localGrid[j] = node1;
                    localGrid[i] = node2;

                    // Recalculate cost to each neighbor based on the node's new position
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){
                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = table1hop[distManhattanI*17 + distManhattanJ];
                            nextCost += increment;
                        }

                    }
                    if(node2!=255){

                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            increment = table1hop[distManhattanI*17 + distManhattanJ];
                            nextCost += increment;
                        }

                    }  

                    // Annealing acceptance criteria
                    double valor = exp(-1*(nextCost - currentCost)/T);
                    rndIndex = (rndIndex>=MAX_RANDS)?tId:rndIndex;
                    cuda_random = d_random[rndIndex];
                    rndIndex++;
  
                    if(nextCost <= currentCost || cuda_random <= valor)
                        currentCost = nextCost;
                    else {
                        if(node1!=255) localPositions[node1] = old1;
                        if(node2!=255) localPositions[node2] = old2;
                        localGrid[j] = node2;
                        localGrid[i] = node1;
                    }
                }

                T*=0.999;
            }
    }

    d_costs[tId] = currentCost;
	for(int i=0; i<gridSize; i++) d_results[t_istart+i] = localGrid[i];
}


void GPU_results(const char* fileName) {

    char placements[1000]; 
    strcpy(placements, fileName);
    strcat(placements, "_SA1hop_GPU_1k.out");

    std::ofstream output(placements, std::ofstream::out);

    int temp1Best[gridSize] = {0};
    int temp2[gridSize] = {0};
    
    int best = 100000;
    int tid = 0;
    int thread = 0;
    int pos = 0;

    for(int i=0; i<numGrids*gridSize; i++){ 

		if(i%gridSize==0 && i!=0) {

            pos = 0;
            if(h_costs[thread] < best) {
                best = h_costs[thread];
                for(int i=0; i<gridSize; i++) temp1Best[i] = temp2[i];
                tid = thread;
            }

            thread++;
            output << "\n";
        }

            temp2[pos++] = h_results[i];
            output << unsigned(h_results[i]) << " ";
    }

    output.close();

    // Best cost found
    std::cout << placements << "\t" << h_costs[tid] << "\t";
    for(int i=0; i<gridSize; i++){

        if(i%dim==0)
            std::cout << "\n" << temp1Best[i] << "\t";
        else
            std::cout << temp1Best[i] << "\t";

    }
    std::cout << "\n";

}

int main(int argc, const char **argv) {

    double timer = 0;
    cudaDeviceProp device;
    cudaGetDeviceProperties(&device, 0);

    // Pre-calculating random numbers (0,1]
	curandGenerator_t gen;
    cudaMalloc(&d_random, sizeof(float) * MAX_RANDS);
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerateUniform(gen, d_random, MAX_RANDS);
    
    h_costs = (int*) malloc(sizeof(int) * MAX_GRIDS);
    h_table1hop_1d = (int*) malloc(sizeof(int) * 17*17);
    h_tablemesh_1d = (int*) malloc(sizeof(int) * 17*17);

    int idx = 0;
    for(int i=0; i<17; i++)
        for(int j=0; j<17; j++){

            h_table1hop_1d[idx] = h_table1hop[i][j];
            h_tablemesh_1d[idx] = h_tablemesh[i][j];

            idx++;

        }

    // Read Graph
    readInput(argv[1]);
    std::cerr << "FILE: " << argv[3] << " running on " << device.name << std::endl;
    
	// Converting vector A to array
	h_A = (int*) malloc(sizeof(int) * A.size()); 
	memcpy(h_A, A.data(), sizeof(int) * A.size());

    // Read it's random placements (1k shuffles)
    readPlacements(argv[2]);

    // Allocating GPU memory
	cudaMalloc(&d_A,   		 sizeof(int)   * A.size());
	cudaMalloc(&d_v,   		 sizeof(int)   * nodes);
	cudaMalloc(&d_v_i, 		 sizeof(int)   * nodes);
    cudaMalloc(&d_seeds,     sizeof(float) * MAX_RANDS);
    cudaMalloc(&d_table1hop, sizeof(int)   * 17*17);
    cudaMalloc(&d_tablemesh, sizeof(int)   * 17*17);

	// Getting data from host to device
	cudaMemcpy(d_A, h_A,                    sizeof(int) * A.size(), cudaMemcpyHostToDevice); 
	cudaMemcpy(d_v, v, 	                    sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_v_i, v_i,                  sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_table1hop, h_table1hop_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_tablemesh, h_tablemesh_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);

	// Device SETUP
	size_t placements_size = MAX_BYTES * MAX_GRIDS; 
	
    // Host variables memory allocation
	h_results =   (int*) malloc(sizeof(int) * placements_size);
	
	// Allocating device memory
    cudaMalloc(&d_results,	sizeof(int) * placements_size); 
	cudaMalloc(&d_grid, 	sizeof(int) * placements_size);
    cudaMalloc(&d_costs, 	sizeof(int) * MAX_GRIDS);
   
   	// Getting data from host random_placements 
    cudaMemcpy(d_grid, h_randomPlacements,  sizeof(int) * placements_size,  cudaMemcpyHostToDevice);
    cudaMemcpy(d_costs, h_costs,            sizeof(int) * MAX_GRIDS,        cudaMemcpyHostToDevice);

   
    std::chrono::time_point<std::chrono::high_resolution_clock> start_clock, end_clock;
    start_clock = std::chrono::high_resolution_clock::now();

    annealing <<< MAX_BLOCKS, MAX_THREADS >>> ( nodes, dim, gridSize, 
                                                d_grid, d_v_i, d_v, d_A, 
                                                d_results, d_costs, d_random,
                                                d_table1hop, d_tablemesh
                                            );

    cudaDeviceSynchronize();

    cudaMemcpy(h_results, d_results,    sizeof(int) * placements_size,  cudaMemcpyDeviceToHost);
    cudaMemcpy(h_costs, d_costs,        sizeof(int) * MAX_GRIDS,        cudaMemcpyDeviceToHost);

    end_clock = std::chrono::high_resolution_clock::now();
    timer = std::chrono::duration_cast<Seconds>(end_clock - start_clock).count();

	GPU_results(argv[3]);
    printf("%.3lf\n", timer);    
}

/* Useful to know runtime specs
std::cerr << endl << "FILE: " << argv[3] << std::endl;
std::cerr << "BLOCKS: " << MAX_BLOCKS << std::endl;
cudaDeviceProp device;
cudaGetDeviceProperties(&device, 0);
printf("DEVICE: %s\nMAX_BLOCKS: %d\nMAX_THREADS: %d\n", device.name, device.maxGridSize[0], device.maxThreadsDim[0]);    
printf("RUNTIME: %.3lfs\n\n", timer);
*/

##Checking GPU data

In [ ]:
!nvcc GPU_SA.cu -lcurand
!mkdir GPU_results 2>/dev/null || true

!echo "Running micro instances"
!for file in nvcc_colab_icpp2020/list_micro/*.in; do ./a.out $file random_placement/micro/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> micro_1hop.txt; done
!mkdir GPU_results/1hop_micro 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_micro

!echo "Running kmeans_exp instances"
!for file in nvcc_colab_icpp2020/list_kmeans_expanded/*.in; do ./a.out $file random_placement/kmeans_exp/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_exp_1hop.txt; done
!mkdir GPU_results/1hop_kmeans_exp 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_kmeans_exp

!echo "Running cgrame instances"
!for file in nvcc_colab_icpp2020/list_cgrame/*.in; do ./a.out $file random_placement/cgrame/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> cgrame_1hop.txt; done
!mkdir GPU_results/1hop_cgrame 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_cgrame

!echo "Running kmeans instances"
!for file in nvcc_colab_icpp2020/list_kmeans/*.in; do ./a.out $file random_placement/kmeans/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_1hop.txt; done
!mkdir GPU_results/1hop_kmeans 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_kmeans

!echo "Running tree instances"
!for file in nvcc_colab_icpp2020/list_tree/*.in; do ./a.out $file random_placement/tree/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> tree_1hop.txt; done
!mkdir GPU_results/1hop_tree 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_tree

!echo "Running map instances"
!for file in nvcc_colab_icpp2020/list_map/*.in; do ./a.out $file random_placement/map/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> map_1hop.txt; done
!mkdir GPU_results/1hop_map 2>/dev/null || true
!mv *SA1hop_GPU_1k.out GPU_results/1hop_map

# CHESS

In [ ]:
%%writefile GPU_SA.cu
#include "core.h"

int h_table1hop[17][17] = 
{{0,  0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15}};

int h_tablemesh[17][17] =
{{0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8},  
 {0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8},  
 {1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9},  
 {1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9},  
 {2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10},  
 {2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10},  
 {3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11},  
 {3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11},  
 {4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12},  
 {4,  5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12},  
 {5,  5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13},  
 {5,  6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13},  
 {6,  6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14},  
 {6,  7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14},  
 {7,  7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15},  
 {7,  8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15},  
 {8,  8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16}};


int gridCost(int *positions){

     int cost_ = 0, increment=0, distManhattanI, distManhattanJ, chess;
    for(int k=0; k<edges; k++){
        int ifrom = positions[h_edgeA[k]]/dim; 
        int jfrom = positions[h_edgeA[k]]%dim; 
        int ito = positions[h_edgeB[k]]/dim; 
        int jto = positions[h_edgeB[k]]%dim;
        distManhattanJ = abs(jto - jfrom);
        distManhattanI = abs(ito - ifrom);

        chess = (abs(jfrom-ifrom)%2==0) ? 1 : 0;
        increment = (chess==1) ? h_table1hop[distManhattanI][distManhattanJ] : h_tablemesh[distManhattanI][distManhattanJ];     
        cost_ += increment;
    }
    return cost_;

}

void readPlacements(const char* fileName) {

    std::string name(fileName); // Without this line, the input mac_inout gives segfault..dunno why
    std::ifstream file(name, std::ifstream::in); 
    if(!file.good()) {
        printf("Failed to load random placements. Check file name or it's existence\nAborting");
        exit(EXIT_FAILURE);
    }

    std::string s;
    int* temp = (int*) malloc(sizeof(int) * gridSize);
    int i = 0;
    int start = 0;
    numGrids = 0;
    while(std::getline(file, s) && numGrids < MAX_GRIDS ) {
        
		std::stringstream line(s);
        int n;
        
        // Read placement line and store in @h_randomPlacements
        while(line >> n) h_randomPlacements[i++] = n;

        // Evaluating cost
        int positions[256];
        for(int k=start, l=0; k<i; k++, l++) {
            temp[l] = h_randomPlacements[k];
            positions[temp[l]] = l;
        }

        // Updating cost for 'ith' grid
        h_costs[numGrids] = gridCost(positions);
        start = i;
        numGrids++;
    }

    file.close();
}


__global__ void annealing(int nodes, int dim, int gridSize,              		// Variables
			   			  	int *d_grid,                        			 	// Initializers 
			   				int *v_i, int *v, int *A,                			// Graph 
                            int *d_costs, float* d_random,   	// Output and random states
                            int *table1hop, int *tablemesh               		// New cost parameters
                         ){		     

    
    // Determinig thread ID and checking if it's within boudaries
    int tId = blockIdx.x * blockDim.x + threadIdx.x;

	if( tId >= MAX_GRIDS ) return; 

    // Which sector of d_grid belongs to the thread
    size_t t_istart = tId*gridSize;
    if(t_istart > MAX_GRIDS*MAX_BYTES) printf("DAYUUUMNNN\n");
    if(tId > MAX_RANDS) printf("WHAAAAAAAAAT\n");
    float cuda_random;

    int currentCost = d_costs[tId], nextCost, increment, distManhattanI, distManhattanJ, chess;
	
    int* localGrid = 		(int*) malloc(sizeof(int) * gridSize);
	int* localPositions = 	(int*) malloc(sizeof(int) * 256);
    
    for(int i=0; i<gridSize; i++) {
        localGrid[i] = d_grid[t_istart + i];
        localPositions[localGrid[i]] = i;
    }

    /* Simulated Annealing */
    unsigned int rndIndex = tId; 	// Each thread starts in a different place in @d_random
    bool foundOptimal = false;
    double T = 100;
    while(T>=0.00001 && !foundOptimal){

        for(int i=0; i<gridSize; i++){

                if(foundOptimal) break;

                for(int j=i+1; j<gridSize; j++){

					if(currentCost==0) {
						foundOptimal = true;
						break;
					}

                    // If we're looking at 2 empty spaces, skip                   
                    if(localGrid[i]==255 && localGrid[j]==255) continue;

                    int node1 = localGrid[i];
                    int node2 = localGrid[j];
                    
					nextCost = currentCost;
                    
					// Remove cost from node to it's neighbors
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){
                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim;
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);

                            if(abs(jfrom-ifrom)%2==0) chess=1;
                            else chess = 0;

                            if(chess==1) increment = table1hop[distManhattanI*17 + distManhattanJ];
                            else increment = tablemesh[distManhattanI*17 + distManhattanJ];

                            nextCost -= increment;
                            
                        }
                        
                    }
                    if(node2!=255){
                        
                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);

                            if(abs(jfrom-ifrom)%2==0) chess=1;
                            else chess = 0;

                            if(chess==1) increment = table1hop[distManhattanI*17 + distManhattanJ];
                            else increment = tablemesh[distManhattanI*17 + distManhattanJ];
    
                            nextCost -= increment;
                        }

                    }

                    // Swap nodes
                    int old1 = i;
                    int old2 = j;

                    if(node1!=255) localPositions[node1] = old2;
                    if(node2!=255) localPositions[node2] = old1;

                    localGrid[j] = node1;
                    localGrid[i] = node2;

                    // Recalculate cost to each neighbor based on the node's new position
                    if(node1!=255){

                        for(int i=0; i<v[node1]; i++){
                            int ifrom = localPositions[node1]/dim; 
                            int jfrom = localPositions[node1]%dim; 
                            int ito = localPositions[A[v_i[node1]+i]]/dim; 
                            int jto = localPositions[A[v_i[node1]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            if(abs(jfrom-ifrom)%2==0) chess=1;
                            else chess = 0;

                            if(chess==1) increment = table1hop[distManhattanI*17 + distManhattanJ];
                            else increment = tablemesh[distManhattanI*17 + distManhattanJ];

                            nextCost += increment;
                        }

                    }
                    if(node2!=255){

                        for(int i=0; i<v[node2]; i++){
                            int ifrom = localPositions[node2]/dim; 
                            int jfrom = localPositions[node2]%dim; 
                            int ito = localPositions[A[v_i[node2]+i]]/dim; 
                            int jto = localPositions[A[v_i[node2]+i]]%dim; 
                            distManhattanJ = abs(jto - jfrom);
                            distManhattanI = abs(ito - ifrom);
    
                            if(abs(jfrom-ifrom)%2==0) chess=1;
                            else chess = 0;

                            if(chess==1) increment = table1hop[distManhattanI*17 + distManhattanJ];
                            else increment = tablemesh[distManhattanI*17 + distManhattanJ];

                            nextCost += increment;
                        }

                    }  

                    // Annealing acceptance criteria
                    double valor = exp(-1*(nextCost - currentCost)/T);
                    rndIndex = (rndIndex>=MAX_RANDS)?tId:rndIndex;
                    cuda_random = d_random[rndIndex];
                    rndIndex++;
  
                    if(nextCost <= currentCost || cuda_random <= valor)
                        currentCost = nextCost;
                    else {
                        if(node1!=255) localPositions[node1] = old1;
                        if(node2!=255) localPositions[node2] = old2;
                        localGrid[j] = node2;
                        localGrid[i] = node1;
                    }
                }

                T*=0.999;
            }
    }

    d_costs[tId] = currentCost;
	for(int i=0; i<gridSize; i++) d_grid[t_istart+i] = localGrid[i];
}


void GPU_results(const char* fileName) {

    char placements[1000]; 
    strcpy(placements, fileName);
    strcat(placements, "_SAchess_GPU_1k.out");

    std::ofstream output(placements, std::ofstream::out);

    int temp1Best[gridSize] = {0};
    int temp2[gridSize] = {0};
    
    int best = 100000;
    int tid = 0;
    int thread = 0;
    int pos = 0;

    for(int i=0; i<numGrids*gridSize; i++){ 

		if(i%gridSize==0 && i!=0) {

            pos = 0;
            if(h_costs[thread] < best) {
                best = h_costs[thread];
                for(int i=0; i<gridSize; i++) temp1Best[i] = temp2[i];
                tid = thread;
            }

            thread++;
            output << "\n";
        }

            temp2[pos++] = h_results[i];
            output << unsigned(h_results[i]) << " ";
    }

    output.close();

    // Best cost found
    std::cout << placements << "\t" << h_costs[tid] << "\t";
    for(int i=0; i<gridSize; i++){

        if(i%dim==0)
            std::cout << "\n" << temp1Best[i] << "\t";
        else
            std::cout << temp1Best[i] << "\t";

    }
    std::cout << "\n";
    

}

int main(int argc, const char **argv) {

    double timer = 0;

    // Pre-calculating random numbers (0,1]
	curandGenerator_t gen;
    cudaMalloc(&d_random, sizeof(float) * MAX_RANDS);
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, 1234ULL);
    curandGenerateUniform(gen, d_random, MAX_RANDS);
    
    h_costs = (int*) malloc(sizeof(int) * MAX_GRIDS);
    h_table1hop_1d = (int*) malloc(sizeof(int) * 17*17);
    h_tablemesh_1d = (int*) malloc(sizeof(int) * 17*17);

    int idx = 0;
    for(int i=0; i<17; i++)
        for(int j=0; j<17; j++){

            h_table1hop_1d[idx] = h_table1hop[i][j];
            h_tablemesh_1d[idx] = h_tablemesh[i][j];

            idx++;

        }

    // Read Graph
    readInput(argv[1]);
    std::cerr << "FILE: " << argv[3] << std::endl;
    
	// Converting vector A to array
	h_A = (int*) malloc(sizeof(int) * A.size()); 
	memcpy(h_A, A.data(), sizeof(int) * A.size());

    // Read it's random placements (1k shuffles)
    readPlacements(argv[2]);

    // Allocating GPU memory
	cudaMalloc(&d_A,   		 sizeof(int)   * A.size());
	cudaMalloc(&d_v,   		 sizeof(int)   * nodes);
	cudaMalloc(&d_v_i, 		 sizeof(int)   * nodes);
    cudaMalloc(&d_seeds,     sizeof(float) * MAX_RANDS);
    cudaMalloc(&d_table1hop, sizeof(int)   * 17*17);
    cudaMalloc(&d_tablemesh, sizeof(int)   * 17*17);

	// Getting data from host to device
	cudaMemcpy(d_A, h_A,                    sizeof(int) * A.size(), cudaMemcpyHostToDevice); 
	cudaMemcpy(d_v, v, 	                    sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_v_i, v_i,                  sizeof(int) * nodes   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_table1hop, h_table1hop_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);
    cudaMemcpy(d_tablemesh, h_tablemesh_1d, sizeof(int) * 17*17   , cudaMemcpyHostToDevice);

	// Device SETUP
	size_t placements_size = MAX_BYTES * MAX_GRIDS; 
	
    // Host variables memory allocation
	h_results =   (int*) malloc(sizeof(int) * placements_size);
	
	// Allocating device memory
	cudaMalloc(&d_grid, 	sizeof(int) * placements_size);
    cudaMalloc(&d_costs, 	sizeof(int) * MAX_GRIDS);
   
   	// Getting data from host random_placements 
    cudaMemcpy(d_grid, h_randomPlacements,  sizeof(int) * placements_size,  cudaMemcpyHostToDevice);
    cudaMemcpy(d_costs, h_costs,            sizeof(int) * MAX_GRIDS,        cudaMemcpyHostToDevice);

   
    std::chrono::time_point<std::chrono::high_resolution_clock> start_clock, end_clock;
    start_clock = std::chrono::high_resolution_clock::now();

    annealing <<< MAX_BLOCKS, MAX_THREADS >>> ( nodes, dim, gridSize, 
                                                d_grid, d_v_i, d_v, d_A, 
                                                d_costs, d_random,
                                                d_table1hop, d_tablemesh
                                            );

    cudaDeviceSynchronize();

    cudaMemcpy(h_results, d_grid,    sizeof(int) * placements_size,  cudaMemcpyDeviceToHost);
    cudaMemcpy(h_costs, d_costs,        sizeof(int) * MAX_GRIDS,        cudaMemcpyDeviceToHost);

    end_clock = std::chrono::high_resolution_clock::now();
    timer = std::chrono::duration_cast<Seconds>(end_clock - start_clock).count();

	GPU_results(argv[3]);
    printf("%.3lf\n", timer);
    
    
    clear();
}

/* Useful to know runtime specs
std::cerr << endl << "FILE: " << argv[3] << std::endl;
std::cerr << "BLOCKS: " << MAX_BLOCKS << std::endl;
cudaDeviceProp device;
cudaGetDeviceProperties(&device, 0);
printf("DEVICE: %s\nMAX_BLOCKS: %d\nMAX_THREADS: %d\n", device.name, device.maxGridSize[0], device.maxThreadsDim[0]);    
printf("RUNTIME: %.3lfs\n\n", timer);
*/

## Checking GPU data

In [ ]:
!nvcc GPU_SA.cu -lcurand
!mkdir GPU_results 2>/dev/null || true

!echo "Running micro instances"
!for file in nvcc_colab_icpp2020/list_micro/*.in; do ./a.out $file random_placement/micro/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> micro_chess.txt; done
!mkdir GPU_results/chess_micro 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_micro

!echo "Running kmeans_exp instances"
!for file in nvcc_colab_icpp2020/list_kmeans_expanded/*.in; do ./a.out $file random_placement/kmeans_exp/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_exp_chess.txt; done
!mkdir GPU_results/chess_kmeans_exp 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_kmeans_exp

!echo "Running cgrame instances"
!for file in nvcc_colab_icpp2020/list_cgrame/*.in; do ./a.out $file random_placement/cgrame/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> cgrame_chess.txt; done
!mkdir GPU_results/chess_cgrame 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_cgrame

!echo "Running kmeans instances"
!for file in nvcc_colab_icpp2020/list_kmeans/*.in; do ./a.out $file random_placement/kmeans/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> kmeans_chess.txt; done
!mkdir GPU_results/chess_kmeans 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_kmeans

!echo "Running tree instances"
!for file in nvcc_colab_icpp2020/list_tree/*.in; do ./a.out $file random_placement/tree/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> tree_chess.txt; done
!mkdir GPU_results/chess_tree 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_tree

!echo "Running map instances"
!for file in nvcc_colab_icpp2020/list_map/*.in; do ./a.out $file random_placement/map/$(basename $file ".in")_1k_rand.out $(basename $file "_zigzag.in") >> map_chess.txt; done
!mkdir GPU_results/chess_map 2>/dev/null || true
!mv *SAchess_GPU_1k.out GPU_results/chess_map